<a href="https://colab.research.google.com/github/omridrori/generative-models/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import  pandas as pd
from sklearn.model_selection import train_test_split

In [34]:
#import xlsxwriter
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sklearn

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV
import pdb
import torch  
import torch.optim as optim
from sklearn.tree import DecisionTreeClassifier
from torch.optim import adam    
from sklearn.model_selection import cross_val_score    
    
    

In [35]:

n_features=16
def softmax(vector):
    e = np.exp(vector)
    return e / e.sum()

def make_data():
    n_samples=int(np.ceil(np.abs(np.random.normal(500,500))))
    n_informative=np.random.randint(0,n_features)
    n_repeated=np.random.randint(0,n_features-n_informative)
    n_redundant=n_features-n_informative-n_repeated
    n_classes=np.random.randint(2,n_informative//3+1)
    n_clusters_per_class=np.random.randint(1,n_informative//n_classes)
    vec=np.random.uniform(size=n_classes)
    weights=softmax(vec)
    flip_y=np.random.exponential(0.1)
    class_sep=np.random.uniform(0.1,5)
    data = make_classification(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_redundant=n_redundant, n_repeated=n_repeated, 
                            n_classes=n_classes, weights=weights, flip_y=flip_y, class_sep=class_sep, 
                            hypercube=True, shift=2, scale=1, shuffle=True, random_state=None)
    scales=np.random.uniform(0.1,1000,size=n_features)
    shifts=np.random.uniform(0.1,1000,size=n_features)
    for i in range(n_features):
        data[0][:,i]=data[0][:,i]*scales[i]+shifts[i]
    return data    

    


In [32]:
import torch.nn as nn
import torch.nn.functional as F


outdim=8
nhyper=3
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, outdim)
        self.fc6=nn.Linear(outdim+nhyper, 32)
        self.fc7=nn.Linear(32, 64)
        self.fc8=nn.Linear(64, 128)
        self.fc9=nn.Linear(128, 32)
        self.fc10=nn.Linear(32, 8)
        self.fc11=nn.Linear(8, 1)

    def forward(self,x, hyperParam):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        x=torch.mean(x,dim=0)
        
        concat=torch.cat((x,hyperParam))
        
        x=F.relu(self.fc6(concat))
        x=F.relu(self.fc7(x))
        x=F.relu(self.fc8(x))
        x=F.relu(self.fc9(x))
        x=F.relu(self.fc10(x))
        x=F.sigmoid(self.fc11(x))
        
        return x
    
     
    
    

 
net = Net()

In [36]:
def makeHyper():
  max_depth=np.random.choice([2, 3, 5, 10, 20])
  min_samples_leaf=np.random.choice([5, 10, 20, 50, 100])
  criterion=np.random.choice(["gini", "entropy"])
  criterionNum=1 if criterion=="gini" else 2
  return max_depth,min_samples_leaf,criterion,criterionNum


In [37]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
los=nn.L1Loss()

In [31]:
SumLoss=0
for epoch in range(1,2000):
    print('************************\n')
    
    
    optimizer.zero_grad()
        #create summary of all the data in embedding space
    try:
      X,y=make_data()
      print('X shape: {}\n'.format(X.shape))
              ################find real score
      max_depth,min_samples_leaf,criterion,criterionNum=makeHyper()
      hyperTensor=torch.tensor([max_depth,min_samples_leaf,criterionNum]).float()
      print("hyper parameters: {}\n".format(hyperTensor))
      model=DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
      scores=cross_val_score(model, X, y, cv=10,n_jobs=6)
      real_score=torch.tensor(scores.mean()).float()
      ## real_score=torch.tensor([model.score(X_test,y_test)]).float()
      
      print('real score: {}\n'.format(real_score))




      X=torch.tensor(X).float()
      score_predicted=net(X,hyperTensor)
      print('predicted score: {}\n'.format(score_predicted))


      loss = los(score_predicted, real_score)
      loss.backward()
      optimizer.step()
              # print statistics
      SumLoss+=loss.item()
      print('epoch number: {},the current loss is {}\n'.format(epoch,loss.data))
      print('mean loss is: {}'.format(SumLoss/epoch))


    except:
        continue  
    


    


    
    
    
    


************************

X shape: (38, 16)

hyper parameters: tensor([ 3., 50.,  2.])

real score: 0.6083333492279053

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 1,the current loss is 0.6083333492279053

mean loss is: 0.6083333492279053
************************

************************

X shape: (1806, 16)

hyper parameters: tensor([20., 10.,  1.])



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


real score: 0.7990546226501465

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 3,the current loss is 0.7990546226501465

mean loss is: 0.4691293239593506
************************

X shape: (475, 16)

hyper parameters: tensor([ 3., 10.,  1.])

real score: 0.6527925729751587

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 4,the current loss is 0.6527925729751587

mean loss is: 0.5150451362133026
************************

X shape: (615, 16)

hyper parameters: tensor([ 5., 50.,  2.])

real score: 0.7997884750366211

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 5,the current loss is 0.7997884750366211

mean loss is: 0.5719938039779663
************************

************************

X shape: (1067, 16)

hyper parameters: tensor([  3., 100.,   2.])



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect 

real score: 0.8528037667274475

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 7,the current loss is 0.8528037667274475

mean loss is: 0.530396112373897
************************

X shape: (965, 16)

hyper parameters: tensor([ 2., 50.,  1.])

real score: 0.5398517847061157

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 8,the current loss is 0.5398517847061157

mean loss is: 0.5315780714154243
************************

************************

************************

X shape: (889, 16)

hyper parameters: tensor([ 3., 50.,  2.])

real score: 0.885265588760376

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 11,the current loss is 0.885265588760376

mean loss is: 0.4670809236439792
************************

************************

X shape: (234, 16)

hyper parameters: tensor([10., 10.,  2.])



...

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect 

real score: 0.9063879251480103

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 1992,the current loss is 0.9063879251480103

mean loss is: 0.45661904926909264
************************

************************

************************

X shape: (253, 16)

hyper parameters: tensor([20., 50.,  1.])

real score: 0.9170769453048706

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 1995,the current loss is 0.9170769453048706

mean loss is: 0.45639209177410395
************************

X shape: (186, 16)

hyper parameters: tensor([ 10., 100.,   2.])

real score: 0.5052631497383118

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 1996,the current loss is 0.5052631497383118

mean loss is: 0.456416576272082
************************

X shape: (289, 16)

hyper parameters: tensor([ 5., 10.,  2.])

real score: 0.43263545632362366

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 1997,the current loss is 0.4326354

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect 

real score: 0.8567788600921631

predicted score: tensor([0.], grad_fn=<ReluBackward0>)

epoch number: 1999,the current loss is 0.8567788600921631

mean loss is: 0.45637663859704425


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
